<a href="https://colab.research.google.com/github/nahubn1/Hybrid-Robot-Navigation-System/blob/main/notebooks/model_prototyping/U-Net-FiLM-Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
COLAB_PROJECT_ROOT = '/content/Thesis_Project'
if not os.path.exists(COLAB_PROJECT_ROOT):
  !git clone https://github.com/nahubn1/Hybrid-Robot-Navigation-System {COLAB_PROJECT_ROOT}
os.chdir(COLAB_PROJECT_ROOT) # Change directory into the project
!git pull # Ensure it's the latest version

Cloning into '/content/Thesis_Project'...
remote: Enumerating objects: 976, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 976 (delta 89), reused 20 (delta 20), pack-reused 785 (from 3)
Receiving objects: 100% (976/976), 506.95 KiB | 29.82 MiB/s, done.
Resolving deltas: 100% (550/550), done.
Already up to date.


In [3]:
!pip install -r environment/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

In [4]:
import sys
# Add src to path
sys.path.append(os.path.join(os.getcwd(), 'src'))

# Define Drive paths
DRIVE_DATA_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/data'
DRIVE_MODELS_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/models'
DRIVE_RESULTS_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/results'

# Link Drive storage to local cloned directories
if not os.path.islink('data'):
    !ln -s {DRIVE_DATA_PATH} data
if not os.path.islink('models'):
    !ln -s {DRIVE_MODELS_PATH} models
if not os.path.islink('results'):
    !ln -s {DRIVE_RESULTS_PATH} results

print("✅ Environment setup complete.")

✅ Environment setup complete.


In [5]:
import torch
import numpy as np
from pathlib import Path
from torch.utils.data import DataLoader
from dnn_guidance.data_loader import PathfindingDataset
from dnn_guidance.model import ResNetFPNFiLM
from dnn_guidance.loss import DiceFocalLoss

# Create a temporary minimal dataset
_temp_root = Path('tmp_sanity_data')
_samples = _temp_root/'samples'
_gt = _temp_root/'gt'
_samples.mkdir(parents=True, exist_ok=True)
_gt.mkdir(parents=True, exist_ok=True)

grid = np.zeros((200, 200), dtype=np.uint8)
grid[0, 0] = 8  # start
grid[-1, -1] = 9  # goal
np.savez(_samples/'sample0.npz', map=grid, clearance=2.0, step_size=8.0, config=np.array([]))
heatmap = np.zeros((200, 200), dtype=np.float32)
np.savez(_gt/'sample0.npz', heatmap=heatmap)

# Build dataset and dataloader
dataset = PathfindingDataset(_samples, _gt)
loader = DataLoader(dataset, batch_size=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNetFPNFiLM().to(device)
criterion = DiceFocalLoss()

(grid_batch, robot_batch), target_batch = next(iter(loader))
grid_batch = grid_batch.to(device)
robot_batch = robot_batch.to(device)
target_batch = target_batch.to(device)

logits = model(grid_batch, robot_batch)
print('Logits shape:', logits.shape)
loss = criterion(logits, target_batch)
print('Loss:', loss.item())

assert logits.shape == (1, 1, 200, 200)
assert loss.dim() == 0

loss.backward()
print('✅ Phase 2 Sanity Check Passed: Model, data pipeline, and loss function are fully integrated and functional.')

Logits shape: torch.Size([1, 1, 200, 200])
Loss: 0.6155796051025391
✅ Phase 2 Sanity Check Passed: Model, data pipeline, and loss function are fully integrated and functional.


In [7]:
import yaml
from pathlib import Path

CONFIG_PATH = 'configs/dnn/resnet_fpn_film_v1_baseline.yaml'
with open(CONFIG_PATH, 'r') as f:
    cfg = yaml.safe_load(f)
print(f'Configuration loaded from {CONFIG_PATH}.')

Configuration loaded from configs/dnn/diffusion_unet_v1_baseline.yaml.


In [8]:
import random
import numpy as np
import torch

seed = cfg.get('seed', 0)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
print(f'Random seed set to {seed}.')

Random seed set to 42.


In [9]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from dnn_guidance.data_loader import PathfindingDataset, _pair_files

samples_dir = DRIVE_DATA_PATH / Path(cfg['samples_dir'])
gt_dir = DRIVE_DATA_PATH / Path(cfg['ground_truth_dir'])
all_pairs = _pair_files(samples_dir, gt_dir)
train_pairs, val_pairs = train_test_split(all_pairs, test_size=cfg['val_split'], random_state=seed)
train_dataset = PathfindingDataset(samples_dir, gt_dir, augment=True)
val_dataset = PathfindingDataset(samples_dir, gt_dir, augment=False)
train_dataset.pairs = train_pairs
val_dataset.pairs = val_pairs
train_loader = DataLoader(train_dataset, batch_size=cfg['batch_size'], shuffle=True, num_workers=cfg['num_workers'])
val_loader = DataLoader(val_dataset, batch_size=cfg['batch_size'], shuffle=False, num_workers=cfg['num_workers'])
print('DataLoaders created.')

DataLoaders created.


In [11]:
from dnn_guidance.model import create_model

model_name = cfg.get('model_name', 'unet_film')
if model_name == 'hr_film_net':
    cfg_file = 'configs/dnn/hr_film_net.yaml'
elif model_name == 'diffusion_unet':
    cfg_file = 'configs/dnn/diffusion_unet.yaml'
elif model_name == 'resnet_fpn_film':
    cfg_file = 'configs/dnn/resnet_fpn_film.yaml'
else:
    cfg_file = 'configs/dnn/unet_film.yaml'
model = create_model(model_name, cfg_file)
device = torch.device(cfg['device'])
model = model.to(device)
print(f'Model {model_name} loaded onto {device}.')

ay zare ken
Model diffusion_unet loaded onto cuda.


In [ ]:
import torch.optim as optim

opt_name = cfg['optimizer']['name']
optimizer_cls = getattr(optim, opt_name)
optimizer = optimizer_cls(model.parameters(), lr=cfg['optimizer']['lr'], weight_decay=cfg['optimizer']['weight_decay'])
scheduler = None
if cfg['scheduler']['name'] == 'CosineAnnealing':
    t_max = cfg['epochs'] - cfg['scheduler']['warmup_epochs']
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=t_max)
print('Optimizer and Scheduler initialized.')

Optimizer and Scheduler initialized.


In [ ]:
from dnn_guidance.loss import DiceFocalLoss

loss_cfg = cfg['loss']
loss_fn = DiceFocalLoss(dice_weight=loss_cfg['dice_weight'], focal_weight=loss_cfg['focal_weight'], focal_gamma=loss_cfg['focal_gamma'])
print('Loss function initialized.')

Loss function initialized.


In [ ]:
from datetime import datetime
from pathlib import Path
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler
from dnn_guidance.trainer import train_one_epoch, validate_one_epoch

# Directories for logging and checkpoints
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_root = DRIVE_RESULTS_PATH / Path(cfg['log_dir'])
log_dir = log_root / f"{cfg['run_name']}_{timestamp}"
log_dir.mkdir(parents=True, exist_ok=True)
writer = SummaryWriter(log_dir)

ckpt_dir = DRIVE_RESULTS_PATH / Path(cfg['checkpoints_dir'])
ckpt_dir.mkdir(parents=True, exist_ok=True)
best_model_path = ckpt_dir / f"{cfg['run_name']}_best_model.pth"
latest_ckpt = ckpt_dir / f"{cfg['run_name']}_latest.pth"

scaler = GradScaler(enabled=cfg.get('use_amp', True))
best_dice = -1.0
epochs_no_improve = 0
start_epoch = 0

# Resume from checkpoint if available
if latest_ckpt.exists():
    ckpt = torch.load(latest_ckpt, map_location=device)
    model.load_state_dict(ckpt['model'])
    optimizer.load_state_dict(ckpt['optimizer'])
    if scheduler and ckpt.get('scheduler'):
        scheduler.load_state_dict(ckpt['scheduler'])
    scaler.load_state_dict(ckpt.get('scaler', {}))
    start_epoch = ckpt.get('epoch', 0) + 1
    best_dice = ckpt.get('best_dice', -1.0)
    epochs_no_improve = ckpt.get('epochs_no_improve', 0)
    print(f"Resumed from epoch {start_epoch}")

/tmp/ipython-input-16-2398197603.py:19: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=cfg.get('use_amp', True))


Resumed from epoch 50


In [ ]:
for epoch in range(start_epoch, cfg['epochs']):
    print(f"--- Epoch {epoch+1}/{cfg['epochs']} ---")
    train_loss = train_one_epoch(model, train_loader, optimizer, loss_fn, device, scaler)
    val_loss, val_dice = validate_one_epoch(model, val_loader, loss_fn, device)
    if scheduler:
        scheduler.step()
    lr = optimizer.param_groups[0]['lr']

    writer.add_scalar('loss/train', train_loss, epoch)
    writer.add_scalar('loss/val', val_loss, epoch)
    writer.add_scalar('dice/val', val_dice, epoch)
    writer.add_scalar('lr', lr, epoch)
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Dice: {val_dice:.4f} | LR: {lr:.6f}")

    improved = val_dice > best_dice
    if improved:
        best_dice = val_dice
        torch.save(model.state_dict(), best_model_path)
        print("New best model found, saving checkpoint...")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f"No improvement for {epochs_no_improve} epochs...")

    torch.save({
        'epoch': epoch,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler.state_dict() if scheduler else None,
        'scaler': scaler.state_dict(),
        'best_dice': best_dice,
        'epochs_no_improve': epochs_no_improve
    }, latest_ckpt)

    if epochs_no_improve >= cfg['early_stop_patience']:
        print('Early stopping triggered.')
        break

writer.close()